In [2]:
import torch
import torchvision
import numpy as np
import os.path as osp
import os

In [104]:
CUDA = torch.cuda.is_available()

In [107]:
# GLOBALS
# vot_path = "../../../vot2016/"
vot_path = "/home/zabulskyy/Datasets/vot2016/"
# yolo_pred_path = "../yolo_predictions/extended"
yolo_pred_path = "/mnt/sshfs/YOLO-tracker/yolo_predictions/extended"
crit_vals = {
    "iou": 1,
    "dist2": 1,
#     "abs_pos": 1,
#     "rel_pos": 1,
    "cc": 1,  # class_correspondence
}
with open("../../yoloTorch/data/coco.names", "r") as f:
    classes = f.read().split("\n")[:-1]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [8]:
def dist2(row1, row2):
    pass

def the_closest(row_to_compare, rows):
    pass

def iou(boxA, boxB):
    pass

def the_most_iou(row_to_compare, rows):
    pass

def interpolate(data):
    pass

def read_gt(vot_class, vot_path=vot_path, force_square=True):
    pass

def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, force_square=True):
    pass

def eval_energy(bb1, bb2, cv=crit_vals):
    pass

def class_correlations(class_probs1, class_probs2):
    pass

def eval_class_corr(cls1, cls2, correlations=cm):
    pass

In [9]:
def dist2(row1, row2):
    x11, y11, x12, y12 = row1[1:5] if len(row1) != 4 else row1[:]
    x21, y21, x22, y22 = row2[1:5] if len(row2) != 4 else row2[:]
    x1, y1 = (x11 + x12) / 2, (y11 + y12) / 2
    x2, y2 = (x21 + x22) / 2, (y21 + y22) / 2
    X = (x1 - x2) ** 2
    Y = (y1 - y2) ** 2
    res = float((X + Y) ** 0.5)
    return res
def the_closest(row_to_compare, rows):
    m, i = np.infty, 0
    for n, row in enumerate(rows):
        d = dist2(row, row_to_compare)
        if d < m:
            m = float(d)
            i = n
    return rows[i], i

In [10]:
def iou(boxA, boxB):
    boxA = boxA[1:5] if len(boxA) != 4 else boxA
    boxB = boxB[1:5] if len(boxB) != 4 else boxB

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    res = interArea / float(boxAArea + boxBArea - interArea)
    return res
def the_most_iou(row_to_compare, rows):
    M, i = 0, 0
    for n, row in enumerate(rows):
        v = iou(row, row_to_compare)
        if v > M:
            M = float(v)
            i = n
    return rows[i], i

In [11]:
def interpolate(data):
    """
    interpolate gaps
    choose one detection among multiple, by picking the closest one by the euclidean distance
    ignore class labels
    :param OUTPUT: tensor
    :param NUM_FRAMES:
    :param CUDA: bool - if cuda is available
    :return: interpolated tensor
    """
    OUTPUT = data["output"]
    NUM_FRAMES = data["num_frames"]
    CUDA = data["CUDA"]

    result = torch.zeros((NUM_FRAMES, 8))
    if CUDA:
        result = result.cuda()

    if OUTPUT[0][0] != 0:
        OUTPUT = fill_first(OUTPUT)

    output_iter = 0
    res_iter = 0
    while res_iter != NUM_FRAMES:
        if (output_iter > len(OUTPUT) - 1):
            # break
            result = fill_last(result, NUM_FRAMES)
            break
        if output_iter == OUTPUT.shape[0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # next frame is interpolated
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # missing detection on some frames, fill with missing
        if OUTPUT[output_iter][0] + 1 < OUTPUT[output_iter + 1][0]:
            subframes = generate_subframes(
                OUTPUT[output_iter], OUTPUT[output_iter + 1])
            result[res_iter] = OUTPUT[output_iter]
            for subframe in subframes:
                res_iter += 1
                result[res_iter] = subframe
            output_iter += 1
            res_iter += 1
            continue

        # this frame contains multiple detections, have to choose one
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0]:
            frame = OUTPUT[output_iter][0]
            to_cut = OUTPUT[OUTPUT[:, 0] == frame]
            # closest, _ = the_closest(result[res_iter - 1], to_cut)
            # result[res_iter] = closest
            closest, _ = the_most_iou(result[res_iter - 1], to_cut)
            result[res_iter] = closest

            output_iter += len(to_cut)
            res_iter += 1

            if output_iter >= OUTPUT.shape[0] - 1:
                break

            # missing detection on some frames, fill with missing
            if OUTPUT[output_iter - 1][0] + 1 < OUTPUT[output_iter][0]:
                subframes = generate_subframes(
                    result[res_iter - 1], OUTPUT[output_iter])
                # result[res_iter] = OUTPUT[output_iter]
                for subframe in subframes:
                    result[res_iter] = subframe
                    res_iter += 1
            continue
    return result


In [12]:
def read_gt(vot_class, vot_path=vot_path, force_square=True):
    gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
    with open(gt_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")] for y in l]
        if force_square:
            l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        return torch.tensor(l)
    
def read_full_gt(vot_path=vot_path, force_square=True):
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    X = list()
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            if force_square:
                l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        X.append(l)
    return torch.tensor(X)

In [13]:
def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, pred_format="csv", first_len=True):
    yolo_pred_file = osp.join(yolo_pred_path, vot_class + "." + pred_format)
    with open(yolo_pred_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        im_len = -1
        if first_len:
            im_len = float(l.pop(0))
        l = [[float(x) for x in y.split(",")] for y in l]            
        return torch.tensor(l), im_len
    

In [14]:
# compute class correlation
def ccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))

    for cls in range(len(classes_probs[0])):
        col = classes_probs[:, cls].view((-1, 1))
        tX = col * classes_probs
        sX = torch.sum(tX, dim=0)
        corr_mat[cls] = sX
    return corr_mat

def corr(col1, col2):
    mean1, mean2 = torch.mean(col1), torch.mean(col2)
    cm1, cm2 = col1 - mean1, col2 - mean2
    top = sum(cm1 * cm2)
    bot = torch.sqrt(sum(cm1 ** 2) * sum(cm2 ** 2))
    return top / bot

# compute class correlation coefficient
def cccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))
    num_classes = len(classes_probs[0])
    for cls1 in range(num_classes):
        col1 = classes_probs[:, cls1].view((-1, 1))
        for cls2 in range(cls1 + 1, num_classes):
            print("{}.{} / {}".format(cls1+1, cls2+1, num_classes))
            col2 = classes_probs[:, cls2].view((-1, 1))
            _corr = corr(col1, col2)
#             _corr = np.corrcoef(col1, col2)
            corr_mat[cls1, cls2] = _corr
    return corr_mat


In [15]:
X = []
pred_format = "csv"
for vot_class in os.listdir(yolo_pred_path):
    yolo_pred_file = osp.join(yolo_pred_path, vot_class)
    with open(yolo_pred_file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l.pop(0)
        im_len = -1
        for i in l:
            X.append([float(x) for x in i.split(",")])
X = torch.tensor(X)
cls_probs = X[:, 6:]
# cm = ccc(cls_probs)

In [92]:
def eval_single_class_corr(cls1 : int, cls2 : int, correlations=cm):
    return float(cm[cls1][cls2])

def eval_whole_class_corr(cls1, cls2, correlations=cm):
    # TODO: consider if this worth it
    # too complicated yet
    pass

In [46]:
def normalise(tensor):
    mean = torch.mean(tensor)
    std = torch.std(tensor)
    return (tensor - mean) / std

In [48]:
def write_csv(matrix, file="correlations.csv"):
    for i in matrix.tolist():
        for j in i:
            print(j, sep=", ", end=", ", file=open(file, "a+"))
        print(file=open(file, "a+"))

In [65]:
def read_csv(file="my_correlations.csv"):
    with open(file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")[:-1]] for y in l]   
        return torch.tensor(l)

In [102]:
class_i = classes.index("orange")
mycm_idxs = torch.sort(mycm[class_i], descending=True)[0]
cm_vals = torch.sort(cm[class_i], descending=True)[0]
cm_idxs = torch.sort(cm[class_i], descending=True)[1]

i = 0
while i != 80:
    print(classes[cm_idxs[i]], "\t\t" , float(cm_vals[i]))
    i += 1
    

orange 		 1.0
apple 		 0.48137083649635315
banana 		 0.08236300200223923
carrot 		 0.03402133285999298
sandwich 		 0.015150247141718864
hot dog 		 0.014234399423003197
sports ball 		 0.010089145042002201
vase 		 0.00922414381057024
donut 		 0.00810171477496624
cup 		 0.00753713957965374
spoon 		 0.006514140870422125
bowl 		 0.006388185080140829
frisbee 		 0.004250752739608288
cake 		 0.00244370405562222
toaster 		 0.0023726338986307383
stop sign 		 0.0020912240725010633
teddy bear 		 0.001880791736766696
hair drier 		 0.0014951156917959452
kite 		 0.0012538599548861384
pizza 		 0.001189374248497188
bear 		 0.0011826545232906938
broccoli 		 0.0007590021123178303
umbrella 		 0.000608559581451118
elephant 		 0.00046243719407357275
mouse 		 0.0003519061428960413
knife 		 0.00021426865714602172
clock 		 0.0001405616640113294
fire hydrant 		 0.00011442590039223433
toothbrush 		 9.031452646013349e-05
fork 		 1.1857632671308238e-05
bottle 		 1.8548705611465266e-06
giraffe 		 -4.043225999339484

In [108]:
def eval_energy(bb1, bb2, cv=crit_vals):
    box1, box2 = bb1[1:5], bb2[1:5]
    cls1, cls2 = bb1[5: ], bb2[5: ]
    class1, class2 = torch.argmax(cls1), torch.argmax(cls2)
    
    _iou = iou(box1, box2)
    _cc = max(0, eval_class_corr(class1, class2))
    _dist2 = dist2(box1, box2)  # probably not
    return cv["iou"] * _iou, cv["cc"] * _cc, cv["dist2"] * _dist2

In [119]:
yp = read_yolo_pred("birds2")
frames_with_detection = sorted(list(set(yp[0][:,0].tolist())))